## Toronto Clustering - Coursera Capstone

##### Importing Necessary Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
print('Geocoders module imported.')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium Library imported.')
print('Libraries imported.')

Geocoders module imported.
Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



ModuleNotFoundError: No module named 'folium'

Import BeautifulSoup to scrape data from Wikipedia Site

In [ ]:
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #toronto clustering raw data from wikipedia
page = urllib.request.urlopen(url)

#import BeautifulSoup library to parse HTML
from bs4 import BeautifulSoup

#parse HTML from wikipedia url
soup = BeautifulSoup(page,"lxml")

Print raw html file

In [ ]:
#print(soup.prettify())

Set table equal to the found table within the html code

In [ ]:
table = soup.find('table', class_='wikitable sortable')
#table

Sort the information into their respective columns if text is present

In [ ]:
A = []
B = []
C = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Set table columns and display Table

In [ ]:
d = {'PostalCode': A,'Borough': B,'Neighborhood': C}
df=pd.DataFrame(data = d)
df

if the data in borough is not equal to "Not assigned" then set it to the dataframe
Group data with the same postal code and separate names with comma, and reset the indices

In [ ]:
df.groupby(['Borough','Neighborhood'])['PostalCode'].apply(','.join).reset_index()

df = df[df.Borough != 'Not assigned\n']
df.reset_index(drop=True)

if data contains neighborhood with not assigned, assign it to the borough name
display the number of rows and columns : (rows,columns)

In [ ]:
df = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df['Neighborhood'] = df['Neighborhood'].str.replace('\n', '')
df['PostalCode'] = df['PostalCode'].str.replace('\n', '')
df['Borough'] = df['Borough'].str.replace('\n', '')
df

In [ ]:
print('(Rows, Columns)')
df.shape

In [ ]:
#Set up Foursquare credentials

In [ ]:
#CLIENT_ID = 'CJGPTSYMG5MZSWUK5YNWOGXKVWG1XMY0AMTXURFBQ5HD0TJU' # your Foursquare ID
#CLIENT_SECRET = 'U0BPF2YTRSUC1CEZJJO1HG20WKR1BYIX3N2XA1DWKMMGWKZK' # your Foursquare Secret
#VERSION = '20180604'
#LIMIT = 30

In [ ]:
df

Import module to convert an address to latitude and longitude values

In [ ]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')
coords.head()

In [ ]:
coords.rename(columns={'Postal Code':'PostalCode'},inplace=True)
coords.head()

In [ ]:
df.head()

In [ ]:
df.merge(coords, left_on='PostalCode',right_on='PostalCode',how='outer')

## Map of Toronto Area

In [ ]:
toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

for lat,lon,bor,neig in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighbourhood']):
    label = '{}, {}'.format(neig, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto)
toronto